# Crypto Challenge Set 2

## 9. Implement PKCS#7 padding

https://cryptopals.com/sets/2/challenges/9

In [1]:
def pkcs7(block,blocksize,pad=b'\x04'):
    return block+max(0,(blocksize-len(block)))*pad

In [2]:
block = b"YELLOW SUBMARINE"
pkcs7(block,20)

b'YELLOW SUBMARINE\x04\x04\x04\x04'

## 10. Implement CBC mode

https://cryptopals.com/sets/2/challenges/10

https://en.wikipedia.org/wiki/Block_cipher_mode_of_operation#Cipher_block_chaining_(CBC)

In [167]:
from Crypto.Cipher import AES

def fixedXOR(a,b):
    return bytes([an^bn for an,bn in zip(a,b)])

def AES_CBC_decrypt(cipher,key):
    aes = AES.new(key, AES.MODE_ECB)
    bsize = len(key)
    IV = bsize*b"\x00"
    blocks = [ cipher[i:i+bsize] for i in range(0,len(cipher),bsize) ]
    plaintext = ""
    for i in range(len(blocks)):
        # decrypt block with AES ECB mode
        dec = aes.decrypt(blocks[i])
        # XOR with IV or previous cipher block
        dec = fixedXOR(dec,IV) if i==0 else fixedXOR(dec,blocks[i-1])
        plaintext += "".join([chr(c) for c in dec])
    return plaintext

In [240]:
from binascii import a2b_base64

key = b"YELLOW SUBMARINE"

with open("input/10.txt") as f:
    cipher = a2b_base64(f.read().replace("\n",""))
    plaintext = AES_CBC_decrypt(cipher,key)
    print(plaintext[0:241])
    
with open('input/plaintext.txt', 'w') as f:
    f.write(plaintext)

I'm back and I'm ringin' the bell 
A rockin' on the mike while the fly girls yell 
In ecstasy in the back of me 
Well that's my DJ Deshay cuttin' all them Z's 
Hittin' hard and the girlies goin' crazy 
Vanilla's on the mike, man I'm not lazy


In [229]:
import binascii

def AES_CBC_encrypt(plaintext,key,IV=None):
    aes = AES.new(key, AES.MODE_ECB)
    bsize = len(key)
    if IV==None:
        IV = bsize*b"\x00"
    plainb = bytes(plaintext.encode()) # convert plaintext to bytes
    if len(plainb)%bsize: # padding to multiple of block size if needed 
        plainb = pkcs7(plainb,len(plainb)+bsize-len(plainb)%bsize)
    blocks = [ plainb[i:i+bsize] for i in range(0,len(plainb),bsize) ]
    ciphbl = []
    cipher = b""
    for i in range(len(blocks)):
        # XOR with IV or previous cipher block
        b = fixedXOR(blocks[i],IV) if i==0 else fixedXOR(blocks[i],ciphbl[i-1])
        # encrypt IVed block with AES ECB mode
        enc = aes.encrypt(b)
        ciphbl.append(enc)
        cipher += enc
    return cipher

In [230]:
cipher = AES_CBC_encrypt(plaintext,key)
plaintext = AES_CBC_decrypt(cipher,key)
print(plaintext[0:241])

I'm back and I'm ringin' the bell 
A rockin' on the mike while the fly girls yell 
In ecstasy in the back of me 
Well that's my DJ Deshay cuttin' all them Z's 
Hittin' hard and the girlies goin' crazy 
Vanilla's on the mike, man I'm not lazy


## 11. An ECB/CBC detection oracle

https://cryptopals.com/sets/2/challenges/11

In [202]:
# Write a function to generate a random AES key; that's just 16 random bytes.
# os.urandom() return a string of size random bytes suitable for cryptographic use.
# https://docs.python.org/3.8/library/os.html#os.urandom

import os

def randAESkey(keylen=16):
    return os.urandom(keylen)

randAESkey()

b'\x80\x9c\x89\xcb\xc8\x82k\xac\xe9\x9b\xc8[\xa5\xc0I2'

In [236]:
# Write a function that encrypts data under an unknown key, e.g. a function that generates a random key 
# and encrypts under it. Under the hood, have the function append 5-10 bytes (count chosen randomly) 
# before the plaintext and 5-10 bytes after the plaintext. Have the function choose to encrypt under 
# ECB 1/2 the time, and under CBC the other half (just use random IVs each time for CBC). 

import os
import random
from Crypto.Cipher import AES

def encryption_oracle(plaintext):
    # generate a 16-bytes random key
    keysize = 16
    key = os.urandom(keysize)
    plainb = bytes(plaintext.encode()) # plaintext in bytes
    # prepend and append bytes
    plainb = os.urandom(random.randint(5,10))+plainb+os.urandom(random.randint(5,10)) 
    # pad the plaintext to a multiple of keysize
    if len(plainb)%keysize:
        plainb = pkcs7(plainb,len(plainb)+keysize-len(plainb)%keysize)
    cipher = ""
    mode = random.randint(0,1)
    if mode==1: # encrypt under ECB
        aes_ecb = AES.new(key, AES.MODE_ECB)
        cipher = aes_ecb.encrypt(plainb)
    else: # encrypt under CBC
        IV = os.urandom(keysize)
        aes_cbc = AES.new(key, AES.MODE_CBC, IV) 
        cipher = aes_cbc.encrypt(plainb)
    return mode, cipher

In [241]:
mode, cipher = encryption_oracle(plaintext)

In [232]:
# Detect the block cipher mode the function is using each time. 

def isAESECB(cipher,blocksize=16):
    blocks = [ cipher[i:i+blocksize] for i in range(0,len(cipher),blocksize) ]
    return len(blocks) - len(set(blocks))

In [259]:
ntot = 10
necb = 0

# This is the poem provided as ciphertext at challenge 10. No problem in guessing the cipher mode with it!
with open("input/plaintext.txt") as f:
    plaintext = f.read().strip("\n")

# This plaintext is too short and with no repetitions, it's impossible to detect ECB mode! 
#plaintext = "the quick brown fox jumps over the lazy dog"

# This is an abtract of an ATLAS paper (a real life text!): 
# it's relatively long, with some repetitions (but not many). Detection does not work :-(
#with open("input/HIGG-2018-51_abstract.txt") as f:
#    plaintext = f.read().strip("\n")

#print(plaintext)

for _ in range(ntot):
    mode, cipher = encryption_oracle(plaintext) # mode is the true encryption used (0 = CBC, 1 = ECB)
    pred = isAESECB(cipher)
    necb += pred
    print(mode,pred)

print(100*necb/ntot)

1 1
0 0
1 1
0 0
0 0
1 1
0 0
1 1
1 1
0 0
50.0
